In [ ]:
%matplotlib notebook
import matplotlib.pyplot as plt

import numpy as np
from numpy.random import normal, random_sample, permutation
from numpy.linalg import norm

def splitsets(data, targets, split=10):
    """Splits samples into train and test sets and reshapes target labels as required."""
    n = data.shape[0]
    permut = permutation(n).astype(int)
    targets = np.array([targets, np.abs(targets-1)]).T
    return zip(*((d[permut[n//split:]], d[permut[:n//split]]) for d in (data, targets)))

def viz(data, labels, progress, accuracy):
    """Function visualizing training progress."""
    if not hasattr(fig, 'viz'): viz.fig = plt.figure('Result')
    viz.fig.suptitle("{:3.2%} done with accuracy {:2.2f}"
                     .format((epoch+1)/epochs, accuracy))
    plt.scatter(*zip(*data), c=labels, cmap='bwr')
    viz.fig.canvas.draw()

# Returns n floats uniformly distributed between a and b.
uniform = lambda a, b, n=1: (b - a) * random_sample((1, n)) + a

In [ ]:
# amount of samples
n = 500

# circle data
radius = 5
r = np.append(uniform(0, radius*.5, n//2), uniform(radius*.7, radius, n//2))
angle = uniform(0, 2*np.pi, n)
xy = np.vstack((r * np.sin(angle), r * np.cos(angle)))
t = np.less(norm(xy, axis=0), radius*.5)
circle = (xy.T, t)

# xor data
padding = .3
x = uniform(-5, 5, n); x[x>padding] += padding; x[x<padding] -= padding
y = uniform(-5, 5, n); y[y>padding] += padding; y[y<padding] -= padding
t = np.less(x*y, 0).flatten()
xor = (np.vstack((x, y)).T, t)

# gauss data
gauss = (np.vstack((normal(2, 1, (n//2, 2)), normal(-2, 1, (n//2, 2)))),
         np.append(np.ones(n//2), np.zeros(n//2)))

# spiral data
def genSpiral(deltaT, n, noise = 0):
    points = np.arange(n)
    r = points / n * 5
    t = 1.75 * points / n * 2 * np.pi + deltaT
    return np.vstack((r * np.sin(t), r * np.cos(t))).T
spiral = (np.vstack((genSpiral(0, n//2), genSpiral(np.pi, n//2))),
          np.append(np.ones(n//2), np.zeros(n//2)))

# visualize
fig = plt.figure('Available Datasets')
ax1 = plt.subplot(221); ax1.set_title('circle')
ax1.scatter(*zip(*circle[0]), c=circle[1], cmap='bwr')
ax2 = plt.subplot(222); ax2.set_title('xor')
ax2.scatter(*zip(*xor[0]), c=xor[1], cmap='bwr')
ax3 = plt.subplot(223); ax3.set_title('gauss')
ax3.scatter(*zip(*gauss[0]), c=gauss[1], cmap='bwr')
ax4 = plt.subplot(224); ax4.set_title('spiral')
ax4.scatter(*zip(*spiral[0]), c=spiral[1], cmap='bwr')
fig.canvas.draw()

In [ ]:
# Choose dataset here
# circle, xor, gauss, spiral
(train_data, train_targets), (test_data, test_targets) = splitsets(*gauss)

In [ ]:
import sys
import tensorflow as tf
x = tf.placeholder(tf.float32, shape=[None, 2], name='input_layer')

W_h = tf.Variable(tf.random_normal([2,8]), name='hidden_weights')
b_h = tf.Variable(tf.zeros([8]), name='hidden_bias')
h = tf.nn.sigmoid(tf.matmul(x,W_h) + b_h, name='hidden_layer')

W_y = tf.Variable(tf.random_normal([8,2]), name='output_weights')
b_y = tf.Variable(tf.zeros([2]), name='output_bias')
y = tf.nn.sigmoid(tf.matmul(h,W_y) + b_y, name='output_layer')

t = tf.placeholder(tf.float32, shape=[None, 2], name='targets')

error = tf.reduce_mean(tf.squared_difference(t,y), name='mean_squared_error')
optimizer = tf.train.GradientDescentOptimizer(0.03).minimize(error, name='gradient_descent')

prediction_check = tf.equal(tf.argmax(y,1), tf.argmax(t,1), name='prediction_check')
accuracy = tf.reduce_mean(tf.cast(prediction_check, tf.float32), name='accuracy_measure')

sess = tf.Session()
sess.run(tf.initialize_all_variables())

epochs = 1000
for epoch in range(epochs):
    split = permutation(len(train_targets))
    batch_targets = train_targets[split[:len(split)//5]]
    batch_data = train_data[split[:len(split)//5]]
    sess.run(optimizer, {x: batch_data, t: batch_targets})
    if (epoch+1)%100 == 0:
        perf = sess.run(accuracy, {x: test_data, t: test_targets})
        predictions = sess.run(y, {x: test_data})
        viz(test_data, predictions[:,0], (epoch+1)/epochs, perf)

fig.suptitle("Training done with accuracy {:2.2f}".format(perf))
perf = sess.run(accuracy, {x: test_data, t: test_targets})
predictions = sess.run(y, {x: test_data})
viz(test_data, predictions[:,0], 1, perf)